# Parkinson's Disease detection with Mel Frequency Cepstral Coefficient (MFCC) audio features using Support Vector Machine (SVM) coded without libraries.

# 1. Import the dependencies.

In [ ]:
import numpy as np
import pandas as pd
import librosa
from librosa import feature
from typing import Tuple
import random
import os

# 2. Load the audios with Librosa, and preprocess them using Peak Amplitude Normalization.
- load_audio will return all the audios with their sampling rate.
- to_positive will turn all values to postive numbers (audio is 1D).
- get_max will return the maximum value in each audio array.
- peak_amplitude_normalize will preprocess audios with this formula 10 ** (peak / 20) / maximum_value.

In [ ]:
def load_audio(file_path) -> Tuple:
    audio, sampling_rate = librosa.load(file_path, sr=44100)
    return audio, sampling_rate

def to_positive(n_array):  # Turn all values to positive values
    for i in range(len(n_array)):
        if n_array[i] < 0:
            n_array[i] = -1 * n_array[i]
    return n_array

def get_max(n_array):  # Get the maximum value
    max_value = n_array[0]
    for i in range(1, len(n_array), 1):
        if n_array[i] > max_value:
            max_value = n_array[i]
    return max_value

def peak_amplitude_normalize(audio_data, peak=-3.0):  # Calculate a scaling factor based on the specific peak value
    n_array = to_positive(audio_data)               # (-3 dB) and multiply the entire audio signal by the scaling factor
    maximum_value = get_max(n_array)
    scaling = 10 ** (peak / 20) / maximum_value
    normalized_audio = audio_data * scaling
    return normalized_audio

# 3. Extract MFCC audio features.
- extract_mfcc uses librosa to extract MFCC features (with number of features 13), for each audio.

In [ ]:
def extract_mfcc() -> Tuple:
    directory = '/content/drive/MyDrive/HYP TRAIN DATA/'
    feats = []
    status = []
    # Getting each audio path from the dataset
    for file_name in os.listdir(directory):
        if file_name.endswith('.wav'):
            if "P" in file_name:
                status.append(1)
            if "C" in file_name:
                status.append(0)
            file_path = os.path.join(directory, file_name)
            audio, sampling_rate = load_audio(file_path)
            preprocessed_audio = peak_amplitude_normalize(audio)
            mfcc = np.mean(librosa.feature.mfcc(y=preprocessed_audio, sr=sampling_rate, n_mfcc=13).T, axis=0)
            feats.append(mfcc)

    feats = np.array(feats)
    status = np.array(status)
    return feats, status

# 4. SVM class with data members learning rate, number of iterations, lambda, weights and bias.
- SVM uses a boundary decision (hyperplane) to separate two classes into 1 and -1.
- The weights and bias are derived using Gradient Descent.

In [ ]:
class SupportVectorMachine:
    def __init__(self, learning_rate, num_iterations, lambda_value, weight, bias):
        self._learning_rate = learning_rate
        self._num_iterations = num_iterations
        self._lambda_value = lambda_value
        self._weight = weight
        self._bias = bias

    def return_learning_rate(self):
        return self._learning_rate

    def get_num_iterations(self):
        return self._num_iterations

    def get_lambda_value(self):
        return self._lambda_value

    def update_weight(self, weight):
        self._weight = weight

    def update_bias(self, bias):
        self._bias = bias

    def get_weight(self):
        return self._weight

    def get_bias(self):
        return self._bias

    def fitting(self, data_points, y_values):
        self._rows, self.cols = data_points.shape
        # Initializing the weight and bias values
        # self._the_weight = np.zeros(self.cols)
        # self._the_weight = []
        #
        # for c in range(self.cols):
        #     self._the_weight.append(random.random())
        #
        # self._the_weight = np.array(self._the_weight)
        # self._the_bias = random.random()
        self._x = data_points
        self._y_values = y_values
        # Implementing Gradient Descent for optimization
        for n in range(self.get_num_iterations()):
            weight_bias = self.increment_weight_bias(self._x, self._y_values, self.get_weight(), self.get_bias())
            # self._the_weight = weight_bias[0]
            # self._the_bias = weight_bias[1]
            self.update_weight(weight_bias[0])
            self.update_bias(weight_bias[1])

    def increment_weight_bias(self, data_points, y_values, weights, bias) -> Tuple:
        # Changing labels
        y_values = np.asarray(y_values)
        for i in range(len(y_values)):
            if y_values[i] <= 0:
                y_values[i] = -1
        y_label = y_values

        # Gradients (dw, db)
        for i in range(len(data_points)):
            constraint = y_label[i] * (np.dot(data_points[i], weights) - bias) >= 1
            if constraint is True:
                dw = 2 * self.get_lambda_value() * weights
                db = 0
            else:
                dw = 2 * self.get_lambda_value() * weights - np.dot(data_points[i], y_label[i])
                db = y_label[i]

        weights = weights - self.return_learning_rate() * dw
        bias = bias - self.return_learning_rate() * db
        weight_bias = (weights, bias)
        return weight_bias

    def estimate(self, data_points):
        results = np.dot(data_points, self.get_weight()) - self.get_bias()
        predicted_labels = np.sign(results)
        for i in range(len(predicted_labels)):
            if predicted_labels[i] <= -1:
                predicted_labels[i] = 0
        return predicted_labels

    def one_estimate(self, data_point):
        result = np.dot(data_point, self.get_weight()) - self.get_bias()
        predicted_label = np.sign(result)
        if predicted_label <= -1:
            predicted_label = 0
        return predicted_label

# 5. Standard scaler and splitting of the data.
- standard_scaler uses means and std deviations to standardize the features.
- split splits the data into training and testing sets.

In [ ]:
def standard_scaler(features):
    # Computation of mean and standard deviation
    means = features.mean(axis=0)
    std_deviations = features.std(axis=0)
    # Standardizing the features
    standardized_features = (features - means) / std_deviations
    standardized_features = np.array(standardized_features)
    return standardized_features

def _random_indexes(array, size, random_state):  # For selecting the indexes for test features
    if size > array:
        raise ValueError(str(size) + " features can't be chosen out of " + str(array))
    random_indexes = []
    random.seed(random_state)
    random_index = random.randrange(0, array, 1)
    random_indexes.append(random_index)

    for _ in range(1, size, 1):
        random_index = random.randrange(0, array, 1)
        while random_index in random_indexes:
            random_index = random.randrange(0, array, 1)
        random_indexes.append(random_index)
    random_indexes = np.array(random_indexes)
    return random_indexes

def split(features, targets, test_size, random_state=42):
    number_of_samples = len(targets)
    t_size = test_size * number_of_samples
    t_size = int(t_size) + 1
    random_indexes = _random_indexes(number_of_samples, t_size, random_state)
    x_training, x_testing, y_training, y_testing = [], [], [], []
    features = list(features)
    targets = list(targets)

    for i in range(len(random_indexes)):
        x_testing.append(features[random_indexes[i]])
        y_testing.append(targets[random_indexes[i]])

    for i in range(len(features)):
        if i in random_indexes:
            pass
        else:
            x_training.append(features[i])
            y_training.append(targets[i])
    x_training, x_testing, y_training, y_testing = np.array(x_training), np.array(x_testing), np.array(y_training), \
        np.array(y_testing)
    return x_training, x_testing, y_training, y_testing

# 6. Metrics
- _confusion_matrix computes true positives, fales positives, true negatives, and false negatives.
- accuracy_score, precision_score, recall_score, f1_score andd confusion_matrix compute accuracy, precison, recall, and f1 scores and confusion matrix.

In [ ]:
def _confusion_matrix(y_testing, y_prediction):
    # Computing confusion matrix
    length_of_labels = len(y_testing)
    true_positive, false_positive, true_negative, false_negative = 0, 0, 0, 0
    for i in range(length_of_labels):
        if y_testing[i] == 1:
            if y_testing[i] == y_prediction[i]:
                true_positive += 1
            else:
                false_positive += 1

        if y_testing[i] == 0:
            if y_testing[i] == y_prediction[i]:
                true_negative += 1
            else:
                false_negative += 1
    return true_positive, false_positive, true_negative, false_negative

def accuracy_score(y_testing, y_preds):
    tp, fp, tn, fn = _confusion_matrix(y_testing, y_preds)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    return accuracy

def precision_score(y_testing, y_preds):
    tp, fp, tn, fn = _confusion_matrix(y_testing, y_preds)
    precision = tp / (tp + fp)
    return precision

def recall_score(y_testing, y_preds):
    tp, fp, tn, fn = _confusion_matrix(y_testing, y_preds)
    tp_fn = tp + fn
    if tp_fn == 0:
        return 0.0
    else:
        recall = tp / tp_fn
        return recall

def f1_score(y_testing, y_preds):
    precision = precision_score(y_testing, y_preds)
    recall = recall_score(y_testing, y_preds)
    precision_recall = precision + recall
    if precision_recall == 0:
        return 0.0
    else:
        f1 = (2 * precision * recall) / precision_recall
        return f1

def confusion_matrix(y_testing, y_preds):
    tp, fp, tn, fn = _confusion_matrix(y_testing, y_preds)
    con_mat = []
    positives = [tp, fp]
    negatives = [fn, tn]
    con_mat.append(positives)
    con_mat.append(negatives)
    return con_mat

# 7. Main function.

In [ ]:
if __name__ == "__main__":
    mfcc, status = extract_mfcc()
    features = standard_scaler(mfcc)
    x_train, x_valid, y_train, y_valid = split(mfcc, status, test_size=0.20)
    #y_test = list(y_test)
    the_weight = []
    for c in range(13):
        the_weight.append(random.random())
    the_weight = np.array(the_weight)
    the_bias = random.random()
    svm = SupportVectorMachine(learning_rate=0.1, num_iterations=10000, lambda_value=0.1, weight=the_weight, bias=the_bias)
    svm.fitting(x_train, y_train)
    predictions = svm.estimate(x_valid)

    # Metrics on the validation data
    accuracy = accuracy_score(y_valid, predictions)
    precision = precision_score(y_valid, predictions)
    recall = recall_score(y_valid, predictions)
    f1 = f1_score(y_valid, predictions)
    confusion_mat = confusion_matrix(y_valid, predictions)

    print('Accuracy score on the validation data: ', accuracy)
    print('Precision score on the validation data: ', precision)
    print('Recall score on the validation data: ', recall)
    print('F1 score on the validation data: ', f1)
    print('The confusion matrix on the validation data: ', confusion_mat)
    mfcc_test = x_valid[7]
    pred = svm.one_estimate(mfcc_test)
    print(pred, ' ', y_valid[7])

Accuracy score on the validation data:  0.25
Precision score on the validation data:  1.0
Recall score on the validation data:  0.25
F1 score on the validation data:  0.4
The confusion matrix on the validation data:  [[3, 0], [9, 0]]
1.0   0
